In [5]:
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('../../..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [6]:
from old_embedding import *

In [7]:
#On récupère les noms, on créé des objets pokemon avec, on récupère les types et leurs one-hot encoding et on a notre dataset.
import json
with open("json/sets.json", "r") as f:
    data = json.load(f)

# 1) Noms
pokemon_names = list(data.keys())
#print(pokemon_names)

In [8]:
from poke_env.environment.pokemon import Pokemon
# 2) transormer en objets
pokemons_list = [Pokemon(gen=4, species=i) for i in pokemon_names]

In [9]:
pokemon_types_list = [obtain_one_hot_vector_type(p) for p in pokemons_list]

In [10]:
class TypeAutoencoder(nn.Module):
    def __init__(self, encoded_size=4):  # taille compressée ici
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(17, 12),
            nn.ReLU(),
            nn.Linear(12, encoded_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoded_size, 12),
            nn.ReLU(),
            nn.Linear(12, 17),
            nn.Sigmoid()  # si tu normalises les entrées
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [13]:
#Entrainement autoencodeur
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 1. Prépare tes données
# Supposons que `vectors` est ta liste de vecteurs one-hot (chacun de taille 18)
vectors_tensor = torch.tensor(pokemon_types_list, dtype=torch.float32)
dataset = TensorDataset(vectors_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 2. Initialise ton modèle, ta loss, ton optimizer
encodeur_type = TypeAutoencoder(encoded_size=4) #Ou 3 ?
criterion = nn.MSELoss()
optimizer = optim.Adam(encodeur_type.parameters(), lr=1e-3)

# 3. Entraînement
epochs = 500
for epoch in range(epochs):
    total_loss = 0.0
    for batch in dataloader:
        inputs = batch[0]  # batch est un tuple (input,)
        optimizer.zero_grad()
        outputs = encodeur_type(inputs)
        loss = criterion(outputs, inputs)  # reconstruction loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")

Epoch 1/500 - Loss: 2.6662
Epoch 2/500 - Loss: 2.5974
Epoch 3/500 - Loss: 2.5325
Epoch 4/500 - Loss: 2.4674
Epoch 5/500 - Loss: 2.4042
Epoch 6/500 - Loss: 2.3383
Epoch 7/500 - Loss: 2.2688
Epoch 8/500 - Loss: 2.1889
Epoch 9/500 - Loss: 2.0995
Epoch 10/500 - Loss: 1.9875
Epoch 11/500 - Loss: 1.8619
Epoch 12/500 - Loss: 1.7072
Epoch 13/500 - Loss: 1.5435
Epoch 14/500 - Loss: 1.3688
Epoch 15/500 - Loss: 1.1930
Epoch 16/500 - Loss: 1.0495
Epoch 17/500 - Loss: 0.9484
Epoch 18/500 - Loss: 0.8755
Epoch 19/500 - Loss: 0.8421
Epoch 20/500 - Loss: 0.8177
Epoch 21/500 - Loss: 0.8090
Epoch 22/500 - Loss: 0.8204
Epoch 23/500 - Loss: 0.8017
Epoch 24/500 - Loss: 0.8017
Epoch 25/500 - Loss: 0.7935
Epoch 26/500 - Loss: 0.7992
Epoch 27/500 - Loss: 0.7847
Epoch 28/500 - Loss: 0.7857
Epoch 29/500 - Loss: 0.7826
Epoch 30/500 - Loss: 0.7864
Epoch 31/500 - Loss: 0.7669
Epoch 32/500 - Loss: 0.7832
Epoch 33/500 - Loss: 0.7836
Epoch 34/500 - Loss: 0.7752
Epoch 35/500 - Loss: 0.7728
Epoch 36/500 - Loss: 0.7728
E

In [14]:
torch.save(encodeur_type.state_dict(), "Utils/embedding/old/type_autoencoder.pth")